# Loading Data

Reference - https://python.langchain.com/docs/tutorials/rag/

In [ ]:
# !git clone https://github.com/CalvQ/RAGnaRec.git
# import os

fatal: destination path 'RAGnaRec' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma
!pip install -U langchain langchain-openai
!pip install sentence-transformers langchain chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
 

In [2]:
!pip install seaborn

In [16]:

# Import packages
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import random
import torch

import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from transformers import pipeline

In [4]:
!pip install pyarrow fastparquet huggingface_hub matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.8 MB/s eta 0:00:00


In [5]:
splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet', 'test': 'yelp_review_full/test-00000-of-00001.parquet'}
df_train = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["train"])
df_test = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
df = pd.concat([df_train, df_test], ignore_index=True)
index_list = range(len(df))
random.seed(10701)
indices = random.sample(index_list, 1000)
df = df.iloc[indices] # using only 10,000 rows for PoC to make training quicker

# Retrieval Component

*   Find the most relevant cluster based on the user review
*   Load only the most relevant cluster as the document base for retrieval
*   Use a retriever to find the top k relevant reviews





In [7]:
def get_topic(user_review):
  return 0

user_review = "Great Italian restaurant with excellent pasta"
topic = get_topic(user_review)

# with open(f'clustering/clusters/topic{topic}.cluster', 'rb') as file:
#     vocab_list = pickle.load(file)

#df = pd.DataFrame([vocab_list])
loader = DataFrameLoader(df, page_content_column = "text")

In [8]:
# load documents
docs = loader.load()
# split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents = splits, embedding = embeddings)
retriever = vectorstore.as_retriever(search_type = 'similarity')

<ipython-input-9-3c1f5153597d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# HuggingFace pipline
llm_pipeline = pipeline("text-generation", model = "distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
prompt = "Great Italian restaurant with excellent pasta"
retrieved_docs = retriever.get_relevant_documents(prompt)
user_review = prompt

<ipython-input-11-cba8edbc44d0>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(prompt)


In [12]:
# Display retrieved results
context = ''
print("Retrieved reviews:")
for i, doc in enumerate(retrieved_docs[:5]):
    print(f"Review {i+1}:\n{doc.page_content}\n")
    context += f"Review {i+1}:\n{doc.page_content}\n"

Retrieved reviews:
Review 1:
average italian fare in the heart of verona.\n\ni used to go here purely for the fact that they served pasta on pasta.  you could get meat or cheese tortellini or ravioli served on a bed of spaghetti -- and i love that idea!\n\ni went back recently for dinner and their had been a few changes.\nthey had added some booths (thumbs up) and the menu was different.\ni think there must be a wider selection (the menu now was in book form -- previously, it was a single sheet of paper) but most drastically, the tortellinis/raviolis were not served on a bed of pasta!!\n(also, i do believe the prices went up.  they are definitely higher than the prices listed on their website -- although they may be lunch prices.. . . )\n\ni ended up getting the chicken parm and it was pretty decent.  nothing to write home about -- but pretty good.  my boy got the rib eye (the special of the night) and he said it was to his liking.  his came with garlic mashed potatoes that were good (

In [ ]:
response_prompt = (
    "Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. "
    "The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n"
    f"{retrieved_docs}\n\n"  # Include retrieved reviews for context
    "Please generate a clear and friendly recommendation."
)

In [ ]:
generated_response = llm_pipeline(response_prompt, max_new_tokens=50, num_return_sequences=1)
print("Generated recommendation:")
print('Generated response: ', generated_response[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated recommendation:
Generated response:  Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:

[Document(metadata={'label': 1}, page_content='Consequently, one MUST sit in the dining area for the corresponding restaurant. \\n\\nNow onto the food.  \\n\\nThe wine list is extensive but only includes Italian selections.  Most of the servers are knowledgeable and can recommend pairings. \\n\\nThe appetizers are decent but are not worth the ~$10-$15 cost.  \\n\\nI\'ve sampled both the pastas and the pizzas.  The pastas are underwhelming in terms of originality, taste, and size. Entrees come with bread and oil for dipping, but the bread is too salty for my enjoyment.  I will never understand why carb-loaded restaurants serve bread with their offerings.  It seems redundant. \\n\\nPizzas are the restaurant\'s specialty (at a place

In [ ]:
print((generated_response)[0])

{'generated_text': 'Based on the following reviews, generate a concise recommendation for a great Italian restaurant with excellent pasta. The recommendation should mention one restaurant name, the highlights, and why it stands out:\n\n[Document(metadata={\'label\': 1}, page_content=\'Consequently, one MUST sit in the dining area for the corresponding restaurant. \\\\n\\\\nNow onto the food.  \\\\n\\\\nThe wine list is extensive but only includes Italian selections.  Most of the servers are knowledgeable and can recommend pairings. \\\\n\\\\nThe appetizers are decent but are not worth the ~$10-$15 cost.  \\\\n\\\\nI\\\'ve sampled both the pastas and the pizzas.  The pastas are underwhelming in terms of originality, taste, and size. Entrees come with bread and oil for dipping, but the bread is too salty for my enjoyment.  I will never understand why carb-loaded restaurants serve bread with their offerings.  It seems redundant. \\\\n\\\\nPizzas are the restaurant\\\'s specialty (at a pla

# Generation

Look into ChatPrompTemplate to make a custom prompt https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

Models:
Google Gemma [google/gemma-2-2b-it](https://huggingface.co/google/gemma-2-2b-it)

In [13]:


def get_template(user_review, context):

  YELP_PROMPT_TEMPLATE = f"""The following are customer reviews of a business:

  BEGIN REVIEWS

  {context}

  END REVIEWS

  Based on the above reviews, provide a personalized recommendation to a user who wrote this review: "{user_review}".

  Follow the guidelines below when answering the question:
  1. If something is mentioned in multiple reviews, it is important and more likely to be preferred in the answer.
  2. If the reviews are of a restaurant, bar, or any food establishment, use specific names of dishes, drinks, and desserts.
  3. If the user mentions something that is mentioned in the above reviews, address it in your recommendation.
  """


  return (YELP_PROMPT_TEMPLATE)


In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `fafasdasda` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fafasda

In [ ]:
# from langchain import HuggingFaceHub
# from langchain import PromptTemplate, LLMChain


# prompt = PromptTemplate(template=YELP_PROMPT_TEMPLATE, input_variables=["context", "user_review"])

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()
# llm_chain = LLMChain(prompt=prompt,
#                      llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10, "max_length": 100}))

# question = "Great Italian restaurant with excellent pasta"
# response = llm_chain.run(user_review=user_review, context=context)
# print(response)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device,
)


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# **Testing End-to-End Conversational Review Generation**

User review --> Find similar reviews (context) --> Fill in template with review, context --> Pass to model

In [19]:
def get_prompt(user_review):
  retrieved_docs = retriever.get_relevant_documents(user_review)
  context = ''
  for i, doc in enumerate(retrieved_docs[:5]):
      context += f"Review {i+1}:\n{doc.page_content}\n"
  return get_template(user_review, context)

good_review = 'Great Italian restaurant with excellent pasta'
# bad_review = "I hate Italian food. I hate pizza. Cheese stinks. Pasta is disgusting. The service at the Italian restaurant sucked. I wish I never have to eat Italian food in my life again."
messages = [
    {"role": "user", "content": get_prompt(good_review)},
]

outputs = pipe(messages, max_new_tokens=256)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
print(assistant_response)


Based on your review, "Great Italian restaurant with excellent pasta," and the reviews provided, I think you'll be disappointed at Il Pizzaiolo.  

While the restaurant is praised for its pasta, the reviews highlight some significant issues that might make your experience less than stellar.  

**Here's what you might want to consider:**

* **The menu is limited:**  The reviews mention the menu is now in a book format, which might be a change you don't like.  
* **The pasta is not as good as you remember:**  The reviewer who loved the pasta is disappointed by the lack of the pasta-on-pasta concept. 
* **The prices are high:**  The reviewer mentions that the prices have gone up, and the website prices may not reflect the actual prices. 
* **The service is not great:**  The reviewer mentions a "disrespectful crew" and a "crude and basic" experience. 

**Here are some alternatives you might enjoy:**

* **The Church Brewery:**  The reviewer mentions this as a better option for pizza and amb

### Experimenting with Different Templates

In [24]:
def get_template(user_review, context):
    YELP_PROMPT_TEMPLATE = f"""Here are customer reviews similar to a user review:

BEGIN REVIEWS

{context}

END REVIEWS

Here is the user review: "{user_review}".

You will act as a recommendation engine. Your task is to analyze the user review and provide thoughtful responses based on the reviews provided. Follow these guidelines:

1. If the user is reviewing a business:
  - Recognize their positive or negative experience.
  - Highlight 2-3 relevant points that other reviews support.
  - Suggest 1-2 actionable tips or features to enhance their experience.

2. If the user seeks a recommendation:
  - Respond in a friendly tone, acknowledging their interests or concerns.
  - Recommend 2-3 businesses or features supported by the provided reviews.
  - Briefly explain why each recommendation suits their needs.

General requirements:
  - Base all statements on the provided reviews. Avoid making unsupported claims.
  - Use phrases like "users often mention..." for summarizing trends.
  - End with a friendly, encouraging closing statement.

"""

    return YELP_PROMPT_TEMPLATE

In [21]:
pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device=device,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
from IPython.display import display, Markdown, Latex

In [25]:
def get_prompt(user_review):
  retrieved_docs = retriever.get_relevant_documents(user_review)
  context = ''
  for i, doc in enumerate(retrieved_docs[:20]):
      context += f"Review {i+1}:\n{doc.page_content}\n"
  return get_template(user_review, context)

good_review = 'I really enjoyed eating at Noodle Head in Pittsburgh. The Pad Thai was amazing. My only issue was the long wait time. Can you find me similar restaurants?'
# bad_review = "I hate Italian food. I hate pizza. Cheese stinks. Pasta is disgusting. The service at the Italian restaurant sucked. I wish I never have to eat Italian food in my life again."
messages = [
    {"role": "user", "content": get_prompt(good_review)},
]

outputs = pipe(messages, max_new_tokens=256)

assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
display(Markdown(assistant_response))


It sounds like you had a great experience with Noodle Head, and the Pad Thai definitely sounds delicious!  Users often mention the food quality and atmosphere at Noodle Head, and it seems like you were impressed with the Pad Thai.  

Based on your review and the other reviews, here are a couple of places you might enjoy:

* **Gilleys:**  Users often mention the spacious and airy atmosphere, which sounds like a good fit for your preference.  They also frequently praise the reasonable prices and good food, especially the BBQ and Mac and Cheese.
* **The Commoner:**  This restaurant is known for its delicious food and friendly atmosphere.  Users often mention the quality of the food and the welcoming staff, which could be a good fit for your experience.

I hope this helps you find your next great meal in Pittsburgh!  Have a great time exploring the city's culinary scene.

### Stratified Reviews for Evaluation

In [30]:
good_reviews_good_intent = [
    "I had a fantastic experience at Bright Smiles Dental. The staff was friendly, and the cleaning was thorough. Can you recommend other dentists with great service?",
    "The yoga class at Zen Studio was so relaxing, and the instructor was excellent. Are there similar studios I should check out?",
    "I loved the selection and deals at The Book Nook. Can you recommend other bookstores with a similar vibe?"
]

good_reviews_bad_intent = [
    "The daycare at Little Wonders is great, but I’d like to avoid other centers with long waitlists and inflexible hours. Suggestions?",
    "The optometrist at VisionWorks was excellent, but I found the glasses overpriced. Can you help me avoid other expensive places?",
    "I really enjoyed my stay at Mountain View Lodge, but their check-in process was frustrating. Can you suggest places without similar issues?"
]

bad_reviews_good_intent = [
    "The doctor at CarePoint Clinic was dismissive and rushed through my appointment. Can you recommend a more attentive healthcare provider?",
    "I didn’t enjoy my haircut at Style Haven. The stylist didn’t listen to what I wanted. Any salons with more customer-focused service?",
    "The bakery on Fifth Avenue had stale pastries and no seating. Any good bakeries where I can enjoy fresh treats comfortably?"
]

bad_reviews_bad_intent = [
    "The food at Bob’s Diner was terrible, and the staff was rude. Can you tell me other places to avoid?",
    "The hotel I stayed at, Urban Retreat, had bedbugs and rude staff. What other hotels should I avoid?",
    "The home cleaning service from Sparkle Shine was unreliable and didn’t clean thoroughly. Where else should I steer clear of?"
]

In [28]:
def process_reviews(review_list, list_name):
  print(f"Processing reviews for {list_name}:")
  for review in review_list:
    print(f"\nUser review: {review}\n")
    messages = [
        {"role": "user", "content": get_prompt(review)},
    ]
    outputs = pipe(messages, max_new_tokens=256)
    assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
    display(Markdown(f"**Assistant Response:**\n\n{assistant_response}"))


In [31]:
process_reviews(good_reviews_good_intent, "Good Reviews, Good Intent")
process_reviews(good_reviews_bad_intent, "Good Reviews, Bad Intent")
process_reviews(bad_reviews_good_intent, "Bad Reviews, Good Intent")
process_reviews(bad_reviews_bad_intent, "Bad Reviews, Bad Intent")

Processing reviews for Good Reviews, Good Intent:

User review: I had a fantastic experience at Bright Smiles Dental. The staff was friendly, and the cleaning was thorough. Can you recommend other dentists with great service?



**Assistant Response:**

This is a great review! It sounds like you had a really positive experience at Bright Smiles Dental.  

Users often mention the friendly staff and thorough cleanings as key factors in their positive experiences.  It sounds like you found the same!  

Based on your review, you might also enjoy checking out Dr. Dachs and his team at [Dentist Name].  Users often mention that Dr. Dachs and his team make even the most anxious patients feel comfortable and cared for.  

If you're looking for a dentist who prioritizes patient comfort and communication, Dr. Dachs and his team at [Dentist Name] might be a great fit. 


Let me know if you have any other questions or need more recommendations!


User review: The yoga class at Zen Studio was so relaxing, and the instructor was excellent. Are there similar studios I should check out?



**Assistant Response:**

This is a great question! It sounds like you're looking for a relaxing and well-taught yoga experience.  

Based on your review, it seems like you really enjoyed the instructor and the overall atmosphere of the Zen Studio.  Users often mention that the instructors at Zen Studio are excellent and create a very relaxing environment.  

Here are a couple of studios that might be a good fit for you:

* **[Studio Name]:**  Users have praised the instructors at [Studio Name] for their expertise and ability to create a welcoming and supportive atmosphere. They also often mention the studio's clean and modern facilities. 
* **[Studio Name]:**  [Studio Name] is another great option for those seeking a relaxing yoga experience. Users often mention the studio's focus on mindfulness and the calming ambiance. 

I hope this helps you find your next favorite yoga studio!


User review: I loved the selection and deals at The Book Nook. Can you recommend other bookstores with a similar vibe?



...

**Assistant Response:**

It sounds like you had a really frustrating experience at Bob's Diner!  Based on the reviews, it seems like the food quality and service were major issues.  Users often mention that the food isn't always up to par, and the service can be slow and inattentive.  

Here are a couple of places you might want to check out:

* **[Local BBQ Joint]**:  Users often mention that this place has great BBQ, and the service is friendly and efficient.  They also highlight that the food is consistently good, so you can be sure you'll get a satisfying meal.
* **[Casual Diner]**:  This type of diner often gets positive reviews for its friendly atmosphere and quick service.  Users often mention that the food is simple but delicious, and the prices are reasonable.


I hope you find a place that meets your expectations!  Happy eating!


User review: The hotel I stayed at, Urban Retreat, had bedbugs and rude staff. What other hotels should I avoid?



**Assistant Response:**

It sounds like you had a really frustrating experience at Urban Retreat.  Users often mention bedbugs and rude staff as major red flags, and it's definitely not a good sign.  

Based on the reviews you provided, here are a couple of hotels you might want to consider:

* **The Vdara:**  Users who enjoyed their stay at The Vdara often mention the comfortable rooms, friendly staff, and good amenities.  It seems like you're looking for a hotel with a good balance of comfort and service, and The Vdara seems to deliver on both.
* **Hotels with good reviews for cleanliness:**  Many reviewers mentioned the cleanliness of the rooms at The Vdara.  You might want to look for hotels with consistently high ratings for cleanliness, as that can be a big factor in a positive experience.

I hope this helps you find a hotel that meets your needs.  Remember, it's always a good idea to read reviews carefully before booking, and don't be afraid to ask questions if you're unsure about something.  Happy travels!


User review: The home cleaning service from Sparkle Shine was unreliable and didn’t clean thoroughly. Where else should I steer clear of?



**Assistant Response:**

It sounds like you're having a frustrating experience with Sparkle Shine.  Based on the reviews, it seems like reliability and thoroughness are key concerns.  

Users often mention that Elite is reliable and does a great job, leaving no residues.  They also highlight the efficiency and friendliness of Justin, who provides excellent service.  If you're looking for a similar experience, you might want to consider Elite for your home cleaning needs. 

Another option is to check out [Name of a local cleaning service] - users often mention that they are very thorough and do a great job cleaning your home.  They also have a great reputation for customer service and are known for their friendly and professional staff. 

Remember, it's always a good idea to read reviews and compare different options before making a decision.  Good luck finding the perfect cleaning service for you!